# Lettura valori dei bilanci da files CSV zippati

I dati dei bilanci sono contenuti in dei files CSV all'interno di un singolo ZIP (PCOX2016.zip).
I files sono distribuiti in un albero di directories e possono essere letti direttamente in Pandas, con il seguente algoritmo.

In [1]:
import io
import zipfile

import pandas as pd

with zipfile.ZipFile('/Users/gu/Workspace/open-bilanci-armonizzati/resources/data/PCOX2016.zip') as zfile:
    for name in zfile.namelist():
        if '_MODELLO' in name:
            continue
        items = name.split("/")
        if len(items) == 4:
            _, region, province, file = items
            if province not in ('CHIETI',):
                continue
            if '_dati' in file:
                print("Regione: {0}, Provincia: {1}, file: {2}".format(region, province, file))
                csv = zfile.read((name))
                csv_df = pd.read_csv(
                    io.BytesIO(csv),
                    sep='#',
                    header=None,
                    low_memory=True,
                    dtype=object,
                    keep_default_na=False,
                )
                del csv_df[0]
                del csv_df[8]
                csv_df.columns = [
                    'anno', 'codice_ente', 'tipo_modello', 'q_id', 'v_id', 'c_id', 'valore',
                ]


Regione: ABRUZZO, Provincia: CHIETI, file: CHIETI_dati.txt


I valori non numerici (date) e quelli che hanno valore 0 sono rimossi.

In [2]:
csv_df = csv_df[pd.to_numeric(csv_df.valore, errors='coerce').notnull()]
csv_df = csv_df[csv_df.valore.astype(float) > 0]

Le colonne quadro e voce devono essere trasformate in interi, per essere confrontabili con gli ID inseriti nel DB.
Poi viene costruito l'identificativo `qvc`, che sarà usato per il merge con i dati della struttura del modello, provenienti dal DB.

I valori del primo quadro, tranne il dato del `Fondo di cassa` vengono rimossi.

In [3]:
csv_df['q_id'] = csv_df['q_id'].apply(int)
csv_df['v_id'] = csv_df['v_id'].apply(int)
csv_df['qvc'] = csv_df['q_id'].astype(str) + "." + csv_df['v_id'].astype(str) + "." + csv_df['c_id'].astype(str)
del csv_df['q_id']
del csv_df['v_id']
del csv_df['c_id']
csv_df = csv_df[~csv_df.qvc.isin(['1.4.1', '1.4.0', '1.5.0', '1.5.1', '1.6.0', '1.6.1'])]

In [4]:
csv_df[0:10]

,anno,codice_ente,tipo_modello,valore,qvc
3,2016,4130230010,PCOX,844794.28,1.10.1
6,2016,4130230010,PCOX,41810.28,2.1.3
8,2016,4130230010,PCOX,133575.97,2.1.2
10,2016,4130230010,PCOX,260611.11,2.2.3
13,2016,4130230010,PCOX,95238.69,2.2.2
17,2016,4130230010,PCOX,356050,2.3.2
24,2016,4130230010,PCOX,648644.02,2.15.1
25,2016,4130230010,PCOX,1359500,2.15.5
26,2016,4130230010,PCOX,1357500,2.15.3
27,2016,4130230010,PCOX,1359500,2.15.4


Il numero di valori da inserire per una singola provincia è di circa 100K.

In [5]:
len(csv_df)

122500

A questi dati deve essere aggiunta una colonna con i valori **procapite**, che si può calcolare partendo dal numero di **abitanti**, estratto nei dati di contesto a partire da ISTAT (task `import_istat_inhabitants`) e già presenti nel DB.

In [6]:
import psycopg2
db = psycopg2.connect(dbname="open-bilanci-armonizzati", host="localhost", user="postgres")

territorio_df = pd.read_sql(
"""
select t.cod_finloc, c.istat_abitanti from territori_territorio t
    join territori_contesto c on c.territorio_id=t.id
    where c.anno=2016;
""", 
    db
)

In [7]:
territorio_df[:10]

,cod_finloc,istat_abitanti
0,VILLALVERNIA--1010021800,940
1,BULCIAGO--1030980110,2844
2,CALCO--1030980120,5303
3,PEIA--1030121530,1819
4,CAMPAGNOLA-CREMASCA--1030260101,686
5,CORREZZANA--1030490920,2977
6,TORRAZZO--1010960690,214
7,GRANA--1010070560,596
8,CAVIZZANA--2040830501,243
9,MANTELLO--1030770390,765


Alla colonna **cod_finloc** va aggiunta una contenente *esclusivamente* il codice numerico, in modo che questo sia uguale alla colonna **codice_ente** del DataFrame `csv_df` proveniente dai CSV.

In [8]:
territorio_df['descrizione_ente'] = territorio_df['cod_finloc'].apply(lambda x: x.split('--')[0])
territorio_df['codice_ente'] = territorio_df['cod_finloc'].apply(lambda x: x.split('--')[1])

In [9]:
territorio_df[:10]

,cod_finloc,istat_abitanti,descrizione_ente,codice_ente
0,VILLALVERNIA--1010021800,940,VILLALVERNIA,1010021800
1,BULCIAGO--1030980110,2844,BULCIAGO,1030980110
2,CALCO--1030980120,5303,CALCO,1030980120
3,PEIA--1030121530,1819,PEIA,1030121530
4,CAMPAGNOLA-CREMASCA--1030260101,686,CAMPAGNOLA-CREMASCA,1030260101
5,CORREZZANA--1030490920,2977,CORREZZANA,1030490920
6,TORRAZZO--1010960690,214,TORRAZZO,1010960690
7,GRANA--1010070560,596,GRANA,1010070560
8,CAVIZZANA--2040830501,243,CAVIZZANA,2040830501
9,MANTELLO--1030770390,765,MANTELLO,1030770390


A questo punto le tabelle possono essere unite, attraverso un **merge**.

In [10]:
merged_df = pd.merge(csv_df, territorio_df, on='codice_ente')

In [11]:
len(merged_df)

122500

In [12]:
merged_df[:10]

,anno,codice_ente,tipo_modello,valore,qvc,cod_finloc,istat_abitanti,descrizione_ente
0,2016,4130230010,PCOX,844794.28,1.10.1,ALTINO--4130230010,3082,ALTINO
1,2016,4130230010,PCOX,41810.28,2.1.3,ALTINO--4130230010,3082,ALTINO
2,2016,4130230010,PCOX,133575.97,2.1.2,ALTINO--4130230010,3082,ALTINO
3,2016,4130230010,PCOX,260611.11,2.2.3,ALTINO--4130230010,3082,ALTINO
4,2016,4130230010,PCOX,95238.69,2.2.2,ALTINO--4130230010,3082,ALTINO
5,2016,4130230010,PCOX,356050,2.3.2,ALTINO--4130230010,3082,ALTINO
6,2016,4130230010,PCOX,648644.02,2.15.1,ALTINO--4130230010,3082,ALTINO
7,2016,4130230010,PCOX,1359500,2.15.5,ALTINO--4130230010,3082,ALTINO
8,2016,4130230010,PCOX,1357500,2.15.3,ALTINO--4130230010,3082,ALTINO
9,2016,4130230010,PCOX,1359500,2.15.4,ALTINO--4130230010,3082,ALTINO


Alcuni valori darebbero errore nel calcolo dei valori procapite. Sono rimossi, perché riguardano delle date.

In [13]:
merged_df = merged_df[pd.to_numeric(merged_df.valore, errors='coerce').notnull()]

In [14]:
len(merged_df)

122500

A questo punto è possibile il calcolo dei **valori procapite**.

In [15]:
merged_df['valore_procapite'] = merged_df.valore.astype(float) / merged_df.istat_abitanti

In [16]:
merged_df[:10]

,anno,codice_ente,tipo_modello,valore,qvc,cod_finloc,istat_abitanti,descrizione_ente,valore_procapite
0,2016,4130230010,PCOX,844794.28,1.10.1,ALTINO--4130230010,3082,ALTINO,274.105866
1,2016,4130230010,PCOX,41810.28,2.1.3,ALTINO--4130230010,3082,ALTINO,13.565957
2,2016,4130230010,PCOX,133575.97,2.1.2,ALTINO--4130230010,3082,ALTINO,43.340678
3,2016,4130230010,PCOX,260611.11,2.2.3,ALTINO--4130230010,3082,ALTINO,84.559088
4,2016,4130230010,PCOX,95238.69,2.2.2,ALTINO--4130230010,3082,ALTINO,30.901587
5,2016,4130230010,PCOX,356050,2.3.2,ALTINO--4130230010,3082,ALTINO,115.525633
6,2016,4130230010,PCOX,648644.02,2.15.1,ALTINO--4130230010,3082,ALTINO,210.462044
7,2016,4130230010,PCOX,1359500,2.15.5,ALTINO--4130230010,3082,ALTINO,441.109669
8,2016,4130230010,PCOX,1357500,2.15.3,ALTINO--4130230010,3082,ALTINO,440.460740
9,2016,4130230010,PCOX,1359500,2.15.4,ALTINO--4130230010,3082,ALTINO,441.109669


Questi sono i valori da cui partire per generare tutti i riferimenti da aggiungere alla tabella Valore.

La tabella Valore è composta da questi campi:
- anno
- territorio_id
- quadro_id
- voce_id
- colonna_id
- valore
- valore_procapite

Le trasformazioni necessarie per generare gli id che vanno nella tabella sono queste:
- codice_ente -> territorio_id
- (tipo_modello, anno, q_id, v_id, c_id) -> (quadro_id, voce_id, colonna_id)

Dato il numero di record da inserire nella tabella (110x100k=11M), occorre immaginare delle funzioni di mapping che lavorino in memoria, possibilmente aggiungendo alla tabella direttamente gli id.

Per fare questo è possibile prelevare i valori dalle tabelle che descrivono il modello: Quadro, Voce, Colonna.

In [17]:
qvc_df = pd.read_sql(
"""
select q.year_start, q.year_end, q.certificate_model,
  q.q_id || '.' || v.v_id || '.' || c.c_id as qvc
  from bilanci_quadro q
  join bilanci_voce v on q.id=v.quadro_id
  join bilanci_colonna c on q.id=c.quadro_id or q.parent_id=c.quadro_id
  where q.certificate_model = %(cert_model)s and
    q.year_start <= %(year)s and q.year_end >= %(year)s
    and c_id != 0;""", 
    db,
    params={'cert_model': 'PCOX', 'year': 2016}
)

In [18]:
codice_ente = '4130230010'
print(codice_ente)
pre_df = merged_df[merged_df.codice_ente==codice_ente]
print("pre => ", len(pre_df))
post_df = pd.merge(pre_df, qvc_df, on=['qvc'])
print("post => ", len(post_df))

4130230010
pre =>  1264
post =>  1264


In [19]:
qvc_df = qvc_df[~qvc_df.qvc.isin(['1.4.1', '1.4.0', '1.5.0', '1.5.1', '1.6.0', '1.6.1'])]

In [46]:
len(qvc_df)

8321

In [21]:
merged_df = pd.merge(merged_df, qvc_df, on=['qvc'])

In [24]:
merged_df[:10]

,anno,codice_ente,tipo_modello,valore,qvc,cod_finloc,istat_abitanti,descrizione_ente,valore_procapite,year_start,year_end,certificate_model
0,2016,4130230010,PCOX,844794.28,1.10.1,ALTINO--4130230010,3082,ALTINO,274.105866,2016,2016,PCOX
1,2016,4130230020,PCOX,460569.63,1.10.1,ARCHI--4130230020,2184,ARCHI,210.883530,2016,2016,PCOX
2,2016,4130230040,PCOX,492013.94,1.10.1,ARIELLI--4130230040,1141,ARIELLI,431.212918,2016,2016,PCOX
3,2016,4130230070,PCOX,464251.30,1.10.1,BORRELLO--4130230070,343,BORRELLO,1353.502332,2016,2016,PCOX
4,2016,4130230080,PCOX,1594050.72,1.10.1,BUCCHIANICO--4130230080,5213,BUCCHIANICO,305.783756,2016,2016,PCOX
5,2016,4130230100,PCOX,479122.20,1.10.1,CANOSA-SANNITA--4130230100,1378,CANOSA-SANNITA,347.693904,2016,2016,PCOX
6,2016,4130230110,PCOX,225447.36,1.10.1,CARPINETO-SINELLO--4130230110,597,CARPINETO-SINELLO,377.633769,2016,2016,PCOX
7,2016,4130230120,PCOX,163972.22,1.10.1,CARUNCHIO--4130230120,628,CARUNCHIO,261.102261,2016,2016,PCOX
8,2016,4130230140,PCOX,5779,1.10.1,CASALANGUIDA--4130230140,911,CASALANGUIDA,6.343578,2016,2016,PCOX
9,2016,4130230150,PCOX,560326.03,1.10.1,CASALBORDINO--4130230150,6129,CASALBORDINO,91.422097,2016,2016,PCOX


In [23]:
len(merged_df)

122500

Ora occorre effettuare il merge con i dati dei territori, per ricavare il territorio_id corrispondente al codice_finloc.

In [25]:
t_df = pd.read_sql(
"""select id as territorio_id, cod_finloc from territori_territorio where territorio='C';""",
db,)

In [26]:
merged_df = pd.merge(merged_df, t_df, on='cod_finloc')

In [27]:
merged_df[:10]

,anno,codice_ente,tipo_modello,valore,qvc,cod_finloc,istat_abitanti,descrizione_ente,valore_procapite,year_start,year_end,certificate_model,territorio_id
0,2016,4130230010,PCOX,844794.28,1.10.1,ALTINO--4130230010,3082,ALTINO,274.105866,2016,2016,PCOX,5836
1,2016,4130230010,PCOX,41810.28,2.1.3,ALTINO--4130230010,3082,ALTINO,13.565957,2016,2016,PCOX,5836
2,2016,4130230010,PCOX,133575.97,2.1.2,ALTINO--4130230010,3082,ALTINO,43.340678,2016,2016,PCOX,5836
3,2016,4130230010,PCOX,260611.11,2.2.3,ALTINO--4130230010,3082,ALTINO,84.559088,2016,2016,PCOX,5836
4,2016,4130230010,PCOX,95238.69,2.2.2,ALTINO--4130230010,3082,ALTINO,30.901587,2016,2016,PCOX,5836
5,2016,4130230010,PCOX,356050,2.3.2,ALTINO--4130230010,3082,ALTINO,115.525633,2016,2016,PCOX,5836
6,2016,4130230010,PCOX,648644.02,2.15.1,ALTINO--4130230010,3082,ALTINO,210.462044,2016,2016,PCOX,5836
7,2016,4130230010,PCOX,1359500,2.15.5,ALTINO--4130230010,3082,ALTINO,441.109669,2016,2016,PCOX,5836
8,2016,4130230010,PCOX,1357500,2.15.3,ALTINO--4130230010,3082,ALTINO,440.460740,2016,2016,PCOX,5836
9,2016,4130230010,PCOX,1359500,2.15.4,ALTINO--4130230010,3082,ALTINO,441.109669,2016,2016,PCOX,5836


A questo punto l'inserimento nella tabella Valore è immediato.

In [28]:
len(merged_df)

122500